# TOP AND BOTTOM N ROWS PER GROUP

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/20 20:05:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
student_data = [
    ("Alice Johnson", "Math", 85, True, 95),
    ("Alice Johnson", "English", 78, True, 88),
    ("Alice Johnson", "History", 92, True, 97),
    ("Alice Johnson", "Science", 67, True, 80),
    ("Alice Johnson", "Art", 55, False, 70),
    ("Bob Smith", "Math", 72, True, 85),
    ("Bob Smith", "English", 81, True, 90),
    ("Bob Smith", "History", 60, True, 75),
    ("Bob Smith", "Science", 50, False, 65),
    ("Bob Smith", "Art", 58, True, 77),
    ("Charlie Wood", "Math", 93, True, 98),
    ("Charlie Wood", "English", 89, True, 92),
    ("Charlie Wood", "History", 74, True, 84),
    ("Charlie Wood", "Science", 88, True, 91),
    ("Charlie Wood", "Art", 79, True, 87),
    ("Diane Brown", "Math", 61, True, 82),
    ("Diane Brown", "English", 57, False, 68),
    ("Diane Brown", "History", 69, True, 80),
    ("Diane Brown", "Science", 73, True, 85),
    ("Diane Brown", "Art", 62, True, 76),
    ("Eve Davis", "Math", 54, False, 66),
    ("Eve Davis", "English", 48, False, 58),
    ("Eve Davis", "History", 63, True, 77),
    ("Eve Davis", "Science", 70, True, 83),
    ("Eve Davis", "Art", 66, True, 81)
]

student_schema = ["name","subject","mark","approve","attendance"]

In [4]:
df_student = spark.createDataFrame(data=student_data,schema=student_schema)

df_student.show()

+-------------+-------+----+-------+----------+
|         name|subject|mark|approve|attendance|
+-------------+-------+----+-------+----------+
|Alice Johnson|   Math|  85|   true|        95|
|Alice Johnson|English|  78|   true|        88|
|Alice Johnson|History|  92|   true|        97|
|Alice Johnson|Science|  67|   true|        80|
|Alice Johnson|    Art|  55|  false|        70|
|    Bob Smith|   Math|  72|   true|        85|
|    Bob Smith|English|  81|   true|        90|
|    Bob Smith|History|  60|   true|        75|
|    Bob Smith|Science|  50|  false|        65|
|    Bob Smith|    Art|  58|   true|        77|
| Charlie Wood|   Math|  93|   true|        98|
| Charlie Wood|English|  89|   true|        92|
| Charlie Wood|History|  74|   true|        84|
| Charlie Wood|Science|  88|   true|        91|
| Charlie Wood|    Art|  79|   true|        87|
|  Diane Brown|   Math|  61|   true|        82|
|  Diane Brown|English|  57|  false|        68|
|  Diane Brown|History|  69|   true|    

## GroupBy and Rank using Window and Functions

In [5]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number

In [10]:
window_spec = Window.partitionBy("name").orderBy(col("mark").desc())

In [12]:
df_student_v2 = df_student.withColumn("row",row_number().over(window_spec)).orderBy("name","row")
df_student_v2.show()

+-------------+-------+----+-------+----------+---+
|         name|subject|mark|approve|attendance|row|
+-------------+-------+----+-------+----------+---+
|Alice Johnson|History|  92|   true|        97|  1|
|Alice Johnson|   Math|  85|   true|        95|  2|
|Alice Johnson|English|  78|   true|        88|  3|
|Alice Johnson|Science|  67|   true|        80|  4|
|Alice Johnson|    Art|  55|  false|        70|  5|
|    Bob Smith|English|  81|   true|        90|  1|
|    Bob Smith|   Math|  72|   true|        85|  2|
|    Bob Smith|History|  60|   true|        75|  3|
|    Bob Smith|    Art|  58|   true|        77|  4|
|    Bob Smith|Science|  50|  false|        65|  5|
| Charlie Wood|   Math|  93|   true|        98|  1|
| Charlie Wood|English|  89|   true|        92|  2|
| Charlie Wood|Science|  88|   true|        91|  3|
| Charlie Wood|    Art|  79|   true|        87|  4|
| Charlie Wood|History|  74|   true|        84|  5|
|  Diane Brown|Science|  73|   true|        85|  1|
|  Diane Bro

## Get Top N row per group name

In [14]:
df_student_v2.filter(col("row") <=1 ).show()

+-------------+-------+----+-------+----------+---+
|         name|subject|mark|approve|attendance|row|
+-------------+-------+----+-------+----------+---+
|Alice Johnson|History|  92|   true|        97|  1|
|    Bob Smith|English|  81|   true|        90|  1|
| Charlie Wood|   Math|  93|   true|        98|  1|
|  Diane Brown|Science|  73|   true|        85|  1|
|    Eve Davis|Science|  70|   true|        83|  1|
+-------------+-------+----+-------+----------+---+



## Partition by subject with top 1 

In [20]:
window_spec = Window.partitionBy("subject").orderBy(col("mark").desc())
df_student_v3 = df_student.withColumn("row",row_number().over(window_spec)).orderBy("subject","row")
df_student_v3.show()

+-------------+-------+----+-------+----------+---+
|         name|subject|mark|approve|attendance|row|
+-------------+-------+----+-------+----------+---+
| Charlie Wood|    Art|  79|   true|        87|  1|
|    Eve Davis|    Art|  66|   true|        81|  2|
|  Diane Brown|    Art|  62|   true|        76|  3|
|    Bob Smith|    Art|  58|   true|        77|  4|
|Alice Johnson|    Art|  55|  false|        70|  5|
| Charlie Wood|English|  89|   true|        92|  1|
|    Bob Smith|English|  81|   true|        90|  2|
|Alice Johnson|English|  78|   true|        88|  3|
|  Diane Brown|English|  57|  false|        68|  4|
|    Eve Davis|English|  48|  false|        58|  5|
|Alice Johnson|History|  92|   true|        97|  1|
| Charlie Wood|History|  74|   true|        84|  2|
|  Diane Brown|History|  69|   true|        80|  3|
|    Eve Davis|History|  63|   true|        77|  4|
|    Bob Smith|History|  60|   true|        75|  5|
| Charlie Wood|   Math|  93|   true|        98|  1|
|Alice Johns

In [16]:
df_student_v3.filter(col("row") <=1 ).show()

+-------------+-------+----+-------+----------+---+
|         name|subject|mark|approve|attendance|row|
+-------------+-------+----+-------+----------+---+
|Alice Johnson|History|  92|   true|        97|  1|
| Charlie Wood|    Art|  79|   true|        87|  1|
| Charlie Wood|English|  89|   true|        92|  1|
| Charlie Wood|   Math|  93|   true|        98|  1|
| Charlie Wood|Science|  88|   true|        91|  1|
+-------------+-------+----+-------+----------+---+



## Reverse rank to get Bottom N

In [18]:
window_spec = Window.partitionBy("name").orderBy(col("mark").asc())

df_student_v4 = df_student.withColumn("row",row_number().over(window_spec)).orderBy("name","row")
df_student_v4.show()

+-------------+-------+----+-------+----------+---+
|         name|subject|mark|approve|attendance|row|
+-------------+-------+----+-------+----------+---+
|Alice Johnson|    Art|  55|  false|        70|  1|
|Alice Johnson|Science|  67|   true|        80|  2|
|Alice Johnson|English|  78|   true|        88|  3|
|Alice Johnson|   Math|  85|   true|        95|  4|
|Alice Johnson|History|  92|   true|        97|  5|
|    Bob Smith|Science|  50|  false|        65|  1|
|    Bob Smith|    Art|  58|   true|        77|  2|
|    Bob Smith|History|  60|   true|        75|  3|
|    Bob Smith|   Math|  72|   true|        85|  4|
|    Bob Smith|English|  81|   true|        90|  5|
| Charlie Wood|History|  74|   true|        84|  1|
| Charlie Wood|    Art|  79|   true|        87|  2|
| Charlie Wood|Science|  88|   true|        91|  3|
| Charlie Wood|English|  89|   true|        92|  4|
| Charlie Wood|   Math|  93|   true|        98|  5|
|  Diane Brown|English|  57|  false|        68|  1|
|  Diane Bro

In [19]:
df_student_v4.filter(col("row") <=1 ).show()

+-------------+-------+----+-------+----------+---+
|         name|subject|mark|approve|attendance|row|
+-------------+-------+----+-------+----------+---+
|Alice Johnson|    Art|  55|  false|        70|  1|
|    Bob Smith|Science|  50|  false|        65|  1|
| Charlie Wood|History|  74|   true|        84|  1|
|  Diane Brown|English|  57|  false|        68|  1|
|    Eve Davis|English|  48|  false|        58|  1|
+-------------+-------+----+-------+----------+---+



In [21]:
window_spec = Window.partitionBy("subject").orderBy(col("mark").asc())
df_student_v5 = df_student.withColumn("row",row_number().over(window_spec)).orderBy("subject","row")
df_student_v5.show()

+-------------+-------+----+-------+----------+---+
|         name|subject|mark|approve|attendance|row|
+-------------+-------+----+-------+----------+---+
|Alice Johnson|    Art|  55|  false|        70|  1|
|    Bob Smith|    Art|  58|   true|        77|  2|
|  Diane Brown|    Art|  62|   true|        76|  3|
|    Eve Davis|    Art|  66|   true|        81|  4|
| Charlie Wood|    Art|  79|   true|        87|  5|
|    Eve Davis|English|  48|  false|        58|  1|
|  Diane Brown|English|  57|  false|        68|  2|
|Alice Johnson|English|  78|   true|        88|  3|
|    Bob Smith|English|  81|   true|        90|  4|
| Charlie Wood|English|  89|   true|        92|  5|
|    Bob Smith|History|  60|   true|        75|  1|
|    Eve Davis|History|  63|   true|        77|  2|
|  Diane Brown|History|  69|   true|        80|  3|
| Charlie Wood|History|  74|   true|        84|  4|
|Alice Johnson|History|  92|   true|        97|  5|
|    Eve Davis|   Math|  54|  false|        66|  1|
|  Diane Bro

In [22]:
df_student_v5.filter(col("row") <=1 ).show()

+-------------+-------+----+-------+----------+---+
|         name|subject|mark|approve|attendance|row|
+-------------+-------+----+-------+----------+---+
|Alice Johnson|    Art|  55|  false|        70|  1|
|    Eve Davis|English|  48|  false|        58|  1|
|    Bob Smith|History|  60|   true|        75|  1|
|    Eve Davis|   Math|  54|  false|        66|  1|
|    Bob Smith|Science|  50|  false|        65|  1|
+-------------+-------+----+-------+----------+---+

